# Demo of aiohttp-underscore-apis

https://github.com/sakurai-youhei/aiohttp-underscore-apis

It all started from my solo postmortem of an incident.

"How could I mitigate the incident sooner?"—that thought led me to develop this library.

Elasticsearch has provided a set of underscore-prefixed APIs, enabling admins to efficiently troubleshoot various issues.

I want the same APIs.

That's why here are APIs for aiohttp apps that mimic Elasticsearch.

## Installing dependencies

Note: `wait-for-it.sh` is only needed for this demo.

In [1]:
%%shell

pip install aiohttp
pip install git+https://github.com/sakurai-youhei/aiohttp-underscore-apis.git@2025.11.4a1

curl -L https://raw.githubusercontent.com/vishnubob/wait-for-it/master/wait-for-it.sh -o wait-for-it.sh
chmod +x wait-for-it.sh

  Cloning https://github.com/sakurai-youhei/aiohttp-underscore-apis.git (to revision 2025.11.4a1) to /tmp/pip-req-build-nanld4x_
  Running command git clone --filter=blob:none --quiet https://github.com/sakurai-youhei/aiohttp-underscore-apis.git /tmp/pip-req-build-nanld4x_
  Resolved https://github.com/sakurai-youhei/aiohttp-underscore-apis.git to commit 9fb9c2a616c5c889c16f031033af7a761ffbd835
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 2.8 MB/s eta 0:00:00
  Created wheel for aiohttp-underscore-apis: filename=aiohttp_underscore_apis-2025.11.4a1-py3-none-any.whl size=21647 sha256=21983c923f6bbbf71465c5c59c8d476c01175d39260bf9eb0e9eecc6e3af3878
  Stored in directory: /tmp/pip-ephem-wheel-cache-c23yjwu5/wheels/2c/a8/4d/280f8c49a1d326d1c31819efd9e33edfc29af44e48d008a1e6
Successfully built aiohttp-underscore-apis
  % Total    % Received % Xfer

## Sample web application with a critical bug

This sample web application consists of the following three routes:

1. `/healthz` endpoint: Always responds immediately.
2. `/once` endpoint: Acquires one resource and responds in 0.5 seconds.
3. `/twice` endpoint: Acquires two resources and responds in 0.5 seconds.


In [2]:
%%writefile sample_app.py

from asyncio import BoundedSemaphore, sleep

from aiohttp import web


async def healthz(request: web.Request) -> web.Response:
    return web.Response()

async def acquire_once(request: web.Request) -> web.Response:
    semaphore: BoundedSemaphore = request.app["semaphore"]
    async with semaphore:
        await sleep(0.5)
        return web.Response()


async def acquire_twice(request: web.Request) -> web.Response:
    semaphore: BoundedSemaphore = request.app["semaphore"]
    async with semaphore:
        async with semaphore:
            await sleep(0.5)
            return web.Response()


def init_app(*_):
    app = web.Application()
    app["semaphore"] = BoundedSemaphore(3)
    app.router.add_get("/healthz", healthz)
    app.router.add_get("/once", acquire_once)
    app.router.add_get("/twice", acquire_twice)
    return app

Writing sample_app.py


### It works fine except for congestion

Launching the sample web application on a random port. `wait-for-it.sh` is used to wait for the web server to start.

Once `/healthz` is confirmed, requests are made to `/once` and `/twice`.

In [ ]:
%%shell

PORT=`shuf -i 20000-65535 -n 1`

python -m aiohttp.web -P ${PORT} sample_app:init_app &
./wait-for-it.sh localhost:${PORT} -- curl localhost:${PORT}/healthz

curl localhost:${PORT}/once
curl localhost:${PORT}/twice

kill %1

wait-for-it.sh: waiting 15 seconds for localhost:26760
DEBUG:asyncio:Using selector: EpollSelector
======== Running on http://localhost:26760 ========
(Press CTRL+C to quit)
wait-for-it.sh: localhost:26760 is available after 1 seconds
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:34 +0000] "GET /healthz HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:34 +0000] "GET /once HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:35 +0000] "GET /twice HTTP/1.1" 200 111 "-" "curl/7.81.0"
Stopped


## Let's set up aiohttp-underscore-apis

The underscore APIs are supposed to be exposed separately via a Unix socket.

Since sending requests to and receiving responses from Unix sockets requires  access to the file system, they are generally considered secure. Of course, it is technically possible to expose underscore APIs alongside other routes if desired, but in most cases, I believe this private configuration is preferrable and right.

In [4]:
%%writefile sample_app_with_underscore_apis.py

from functools import partial
from aiohttp import web
from aiohttp_underscore_apis import AiohttpUnderscoreApis

import sample_app


def init_app(*_):
    app = sample_app.init_app()

    aiohttp_underscore_apis = AiohttpUnderscoreApis()

    # Configure it to listen on the UNIX domain socket
    aiohttp_underscore_apis.site_factories.append(
        partial(web.UnixSite, path="/tmp/aiohttp-underscore-apis.sock")
    )

    # Attach it to your app
    app.cleanup_ctx.append(aiohttp_underscore_apis.listener)
    app.middlewares.extend(aiohttp_underscore_apis.middlewares)

    return app

Writing sample_app_with_underscore_apis.py


### CAT APIs enable inspection

In [ ]:
%%shell

PORT=`shuf -i 20000-65535 -n 1`

python -m aiohttp.web -P ${PORT} sample_app_with_underscore_apis:init_app &
./wait-for-it.sh localhost:${PORT} -- curl localhost:${PORT}/healthz

echo
echo ==== List of CAT APIs.
curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock 'http://./_cat' | cat
echo ====

echo
echo ==== Help of /_cat/routes.
curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock 'http://./_cat/routes?help' | cat
echo ====

echo
echo ==== 1 request is recorded for GET /healthz.
curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock 'http://./_cat/routes?v&s=method,path' | cat
echo ====

kill %1

wait-for-it.sh: waiting 15 seconds for localhost:27125
DEBUG:asyncio:Using selector: EpollSelector
======== Running on http://localhost:27125 ========
(Press CTRL+C to quit)
wait-for-it.sh: localhost:27125 is available after 1 seconds

==== List of CAT APIs.
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:37 +0000] "GET /healthz HTTP/1.1" 200 111 "-" "curl/7.81.0"
=^.^=
/routes
/routes/{route_id}
/tasks
/tasks/{task_id}
INFO:aiohttp.access: [04/Nov/2025:06:11:37 +0000] "GET /_cat HTTP/1.1" 200 210 "-" "curl/7.81.0"
====

==== Help of /_cat/routes.
INFO:aiohttp.access: [04/Nov/2025:06:11:37 +0000] "GET /_cat/routes?help HTTP/1.1" 200 772 "-" "curl/7.81.0"
-----------------------  --------------------------------------
id                       Internal identifier
handler                  Route handler
name                     Route name
method                   Route HTTP method
path                     Route path
stats.req.active         Number of active requests
stats.req.total       

## Let's start investigating the bug

### `GET /once` tolerates.

In [ ]:
%%shell

PORT=`shuf -i 20000-65535 -n 1`

python -m aiohttp.web -P ${PORT} sample_app_with_underscore_apis:init_app &
./wait-for-it.sh localhost:${PORT} -- curl localhost:${PORT}/healthz

seq 1 5 | xargs -P 5 -I{} curl localhost:${PORT}/once?{}

echo
echo ==== 5 requests are recorded for GET /once.
curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock 'http://./_cat/routes?v&s=stats.req.total:desc&h=*' | head -2
echo ====

kill %1

wait-for-it.sh: waiting 15 seconds for localhost:51416
DEBUG:asyncio:Using selector: EpollSelector
======== Running on http://localhost:51416 ========
(Press CTRL+C to quit)
wait-for-it.sh: localhost:51416 is available after 1 seconds
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:38 +0000] "GET /healthz HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:38 +0000] "GET /once?1 HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:38 +0000] "GET /once?3 HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:38 +0000] "GET /once?2 HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:38 +0000] "GET /once?4 HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:38 +0000] "GET /once?5 HTTP/1.1" 200 111 "-" "curl/7.81.0"

==== 5 requests are recorded for GET /once.
INFO:aiohttp.access: [04/Nov/2025:06:11:39 +0000] "GET /_cat/routes?v&s

### `GET /twice` does not.

In [ ]:
%%shell

PORT=`shuf -i 20000-65535 -n 1`

python -m aiohttp.web -P ${PORT} sample_app_with_underscore_apis:init_app &
./wait-for-it.sh localhost:${PORT} -- curl localhost:${PORT}/healthz

seq 1 5 | xargs -P 5 -I{} curl localhost:${PORT}/twice?{} &
sleep 10

echo
echo ==== Only 3 are processed for GET /twice, and the rest are hanging.
curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock 'http://./_cat/routes?v&s=stats.req.total:desc&h=*' | head -2
echo ====

sleep 10
kill -9 %1  # Graceful kill takes too long, so kill it with SIGTERM.

wait-for-it.sh: waiting 15 seconds for localhost:25812
DEBUG:asyncio:Using selector: EpollSelector
======== Running on http://localhost:25812 ========
(Press CTRL+C to quit)
wait-for-it.sh: localhost:25812 is available after 1 seconds
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:41 +0000] "GET /healthz HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:41 +0000] "GET /twice?1 HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:11:41 +0000] "GET /twice?5 HTTP/1.1" 200 111 "-" "curl/7.81.0"

==== Only 3 are processed for GET /twice, and the rest are hanging.
INFO:aiohttp.access: [04/Nov/2025:06:11:51 +0000] "GET /_cat/routes?v&s=stats.req.total:desc&h=* HTTP/1.1" 200 1443 "-" "curl/7.81.0"
             id  handler                   name    method    path        stats.req.active    stats.req.total    stats.resp.time_avg_1m    stats.resp.time_avg_5m    stats.resp.time_avg_15m
137197642415856  sample_app.acquire_twice      

## How I could mitigate the incident sooner



### Active requests can be forcibly canceled

In [ ]:
%%shell

PORT=`shuf -i 20000-65535 -n 1`

python -m aiohttp.web -P ${PORT} sample_app_with_underscore_apis:init_app &
./wait-for-it.sh localhost:${PORT} -- curl localhost:${PORT}/healthz

seq 1 5 | xargs -P 5 -I{} curl localhost:${PORT}/twice?{} &
sleep 5

# Forcibly cancel hanging requests on the problematic /twice.
ID=`curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock 'http://./_cat/routes?s=stats.req.total:desc&h=id' | head -1`
curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock -X POST "http://./_routes/${ID}/interrupt"

sleep 1

echo
echo ==== No active requests remain.
curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock 'http://./_cat/routes?v&s=stats.req.total:desc&h=*' | head -2
echo ====

kill %1

wait-for-it.sh: waiting 15 seconds for localhost:54218
DEBUG:asyncio:Using selector: EpollSelector
======== Running on http://localhost:54218 ========
(Press CTRL+C to quit)
wait-for-it.sh: localhost:54218 is available after 1 seconds
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:12:02 +0000] "GET /healthz HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:12:02 +0000] "GET /twice?2 HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:12:02 +0000] "GET /twice?3 HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access: [04/Nov/2025:06:12:07 +0000] "GET /_cat/routes?s=stats.req.total:desc&h=id HTTP/1.1" 200 249 "-" "curl/7.81.0"
INFO:aiohttp.access: [04/Nov/2025:06:12:07 +0000] "POST /_routes/136065766407536/interrupt HTTP/1.1" 204 100 "-" "curl/7.81.0"
curl: (52) Empty reply from server
ccurl: (5u2r)l :E m(p5t2y)  rEemppltyy  frreopml ys efrrvoemr 
server

==== No active requests remain.
INFO:aiohttp.access: [04/Nov/2025:06:12:0

### Problematic routes can be overridden

In [ ]:
%%shell

PORT=`shuf -i 20000-65535 -n 1`

python -m aiohttp.web -P ${PORT} sample_app_with_underscore_apis:init_app &
./wait-for-it.sh localhost:${PORT} -- curl localhost:${PORT}/healthz

ID=`curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock 'http://./_cat/routes' | grep GET | grep twice | cut -f1 -d' ' | head -1`

echo
echo ==== Temporarily configure the problematic /twice to return HTTP 503
curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock -X PUT "http://./_routes/${ID}/settings?pretty" -H "Content-Type: application/json" -d '
{
  "transient": {
    "preempt": {
      "status": 503
    }
  }
}
' | cat
echo ====

seq 1 5 | xargs -P 5 -I{} curl localhost:${PORT}/twice?{}

echo
echo ==== All requests were immediately returned with a 503 status.
curl -s --unix-socket /tmp/aiohttp-underscore-apis.sock 'http://./_cat/routes?v&s=stats.req.total:desc&h=*' | head -2
echo ====

kill %1

wait-for-it.sh: waiting 15 seconds for localhost:32981
DEBUG:asyncio:Using selector: EpollSelector
======== Running on http://localhost:32981 ========
(Press CTRL+C to quit)
wait-for-it.sh: localhost:32981 is available after 1 seconds
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:12:09 +0000] "GET /healthz HTTP/1.1" 200 111 "-" "curl/7.81.0"
INFO:aiohttp.access: [04/Nov/2025:06:12:09 +0000] "GET /_cat/routes HTTP/1.1" 200 382 "-" "curl/7.81.0"

==== Temporarily configure the problematic /twice to return HTTP 503
INFO:aiohttp.access: [04/Nov/2025:06:12:09 +0000] "PUT /_routes/139147612977824/settings?pretty HTTP/1.1" 200 296 "-" "curl/7.81.0"
{
    "139147612977824": {
        "transient": {
            "preempt": {
                "status": 503
            }
        }
    }
}====
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:12:09 +0000] "GET /twice?1 HTTP/1.1" 503 128 "-" "curl/7.81.0"
INFO:aiohttp.access:127.0.0.1 [04/Nov/2025:06:12:09 +0000] "GET /twice?4 HTTP/1.1" 503 128 "-" "curl